In [ ]:
import requests

source = requests.get("https://stacdata.s3.amazonaws.com/highway-primary.geojson").json()

In [ ]:
import shapely.geometry

shapely_shapes = [shapely.geometry.shape(fea['geometry']) for fea in source['features']]
aoi_as_multiline = shapely.geometry.MultiLineString(shapely_shapes)

In [ ]:
from pystac_client import Client

URL = 'https://earth-search.aws.element84.com/v0/'

client = Client.open(URL)

In [ ]:
search = client.search(
    max_items = 10,
    collections = "sentinel-s2-l2a-cogs",
    intersects = aoi_as_multiline,
    datetime = '2022-01-01/2022-01-24'
)

In [ ]:
print(f"AOI as Shapely Geometry object from MultiLine(), found {len(list(search.items()))} items")

In [ ]:
import stackstac

ds = stackstac.stack(search.get_all_items(),  epsg=4326, assets=["B04", "B03", "B05"] )

In [ ]:
search.get_all_items()

In [ ]:
ds.groupby("time.date") # "sentinel:grid_square"

In [ ]:
import geopandas as gpd

gdf = gpd.GeoDataFrame.from_features(search.get_all_items().to_dict()['features'], "epsg:4326")

In [ ]:
# Colour features by cloud cover
fig = gdf.plot(column='eo:cloud_cover',
         cmap='viridis',
        edgecolor="black",
        categorical=True,
        aspect="equal",
        alpha=0.5,
        figsize=(6, 12),
        legend_kwds={"loc": "upper left", "frameon": False, "ncol": 1},
         legend=True)

_ = fig.set_title("STAC Query Results By Cloud Cover")

### Associated with the talk for [Let's Talk Spatial](https://www.linkedin.com/company/82530406/)

 — By [Aman Bagrecha](https://amanbagrecha.github.io/)